# Define environement variables and import the data

In [ ]:
# Import external packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import fft, signal
#import mpld3
#mpld3.enable_notebook()

#import sys
#sys.path.append('c:\\users\\arnaud\\documents\\travail\\ml_projects\\bike_sharing\\src\\data_processing')
#sys.path.append('c:\\users\\arnaud\\documents\\travail\\ml_projects\\bike_sharing\\env\\lib\\site_packages')

# Import local packages
from src.data_processing import load_csv_from_zip as lcfz

# Set some display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%matplotlib inline

In [ ]:
train, test = lcfz.read_csv_from_zip('./../../data/input/bike-sharing-demand.zip', ['train.csv', 'test.csv'])

# Look at the data structure

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

### Summary
At first glance, the training set is made of 10886 records with no missing values.<br>
The datetime column should be turned into a pandas.DatetimeIndex.<br>
The count column is the target variable.<br>
The casual and registered are not present in the test data and cannot be used for the ML algo.<br>
There is one nominal variable: Season.<br>
There are 2 binary variables: holiday, workingday.<br>
There is 1 ordinal variable: Weather.<br>
There are 4 continuous variables: temp, atemp, humidity, windspeed.<br>

# Use the datetime column as the index to make data manipulation easier

In [ ]:
train.set_index(pd.to_datetime(train.pop('datetime')), inplace=True)

In [ ]:
train['dayofweek'] = train.index.dayofweek
train['dayofyear'] = train.index.dayofyear
train['month'] = train.index.month
train['year'] = train.index.year
train['hour'] = train.index.hour

train.sample(5)

In [ ]:
train.year.unique()

In [ ]:
train.year.value_counts()

In [ ]:
train['2011'].month.value_counts().sort_index()

For convenience, the date and time are converted into a DatetimeIndex and the hour, day, month and year components are extracted as individual features.

We check that each period roughly has the same amount of data.

## Try to identifiy gaps in the data

In [ ]:
train['delta_t'] = train.index.to_series().diff()
train.head()

In [ ]:
train['delta_t'].fillna(pd.Timedelta('0 hour'))
train[(train['delta_t']>pd.Timedelta('1 hours')) & (train['delta_t']< pd.Timedelta('1 days'))]['delta_t']

Here we just create a column holding the time delta (difference) from one row wrt the previous one. After that, we selected only the rows where this difference was greater than the regular sampling interval but also shorter than an entire day. We can notice that there are only a few records corresponding to this request and that they mostly occur at night. The missing rows probably correspond to hours where the 'count' value was equal to 0.

In [ ]:
train[(train['delta_t']>pd.Timedelta('1 day'))]['delta_t']

This time, we look at the missing rows where the time delta is greater than 1 day. We notice that the 9 to 12 last days of each months are systematically missing. This correspond to the data that has been removed on purpose to create the test set.

In [ ]:
train.describe()

One important thing to notice here is that the min of 'count' is 1 and not 0. Here, I will assume that when the 'delta_t' is only a few hours (only at night as can be seen above), there are no records because the value of 'count' is actually 0. <font color=red>It would be interesting to compare a ML algo with/-out adding rows when 'count' is 0.</font>

One troubling thing in this data set are the values of the temp and atemp features. We can see here that the minimum temperature is positive. Knowing that this bike sharing program was in Washington DC and that winters can get pretty cold there, this seems unlikely to be the actual temperature. (To understand how I figured out that the program was in Washington DC, I suggest you dig in the "holiday" feature...).

# Univariate and Bivariate analysis

Let's now have a look at the features one by one to get a better feeling of what they represent and how can they impact our target variable.

In [ ]:
cat_features = ['season']
ord_featuire = ['weather']
bin_features = ['holiday', 'workingday']
num_features = ['temp', 'atemp', 'humidity', 'windspeed']
other_features = ['casual', 'registered']
target_variable = ['count']

## Categorical features

### Season

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15,5))
train.boxplot(column='casual', by='season', ax=ax[0])
train.boxplot(column='registered', by='season', ax=ax[1])
train.boxplot(column='count', by='season', ax=ax[2])
plt.savefig('./../../figs/boxplot_gb_season.png')
plt.show()

This box plot shows that there is a higher demand during spring and summer probably due to weather conditions (higher temperatures, less rain, ...). We can also notice that there is less variability for 'registered' customers than for 'casual' ones.

### Weather

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15,5))
train.boxplot(column='casual', by='weather', ax=ax[0])
train.boxplot(column='registered', by='weather', ax=ax[1])
train.boxplot(column='count', by='weather', ax=ax[2])
plt.savefig('./../../figs/boxplot_gb_weather.png')
plt.show()

Here we see again that the better the weather, the higher the demand which makes sense. Also, we can notice that for the weather category #4, there is only 1 entry and its value is quite high. <font color=red>We might want to remove this record to avoid biasing our model.</font>

## Numerical features

### Temperature

In [ ]:
train['temp_bin'] = pd.cut(train['temp'], 10)
fig, ax = plt.subplots(2, 2, figsize=(16,9))
train['temp'].plot(linewidth=0, marker='o', markersize=1, ax=ax[0][0])
train.boxplot(column='casual', by='temp_bin', ax=ax[0][1], rot=45)
train.boxplot(column='registered', by='temp_bin', ax=ax[1][0], rot=45)
train.boxplot(column='count', by='temp_bin', ax=ax[1][1], rot=45)
plt.savefig('./../../figs/boxplot_gb_temp.png')
plt.show()

We can observe here an anual modulation corresponding to the seasons. Hot in summer, cold in winter.

There is a clear correlation between the temperature and the bike demand.
For low temperatures, the demand by casual users is almost inexistant while registered users tend to keep using the bikes (seems reasonable).
The demand increases with temperature and actually reaches a maximum for temperature in the range 2-35 degrees (according to the median, mean might be different).

<font color='red'>One other thing to keep in mind is that temperatures are 'always' lower at night and higher during the day! At the same time, people probably tend to use the bikes more during the day than at night for independant reasons. So we might have so amount of colinearity between those features.</font>

In [ ]:
train['atemp'].plot(linewidth=0, marker='o', markersize=1)
plt.savefig('./../../figs/atemp_vs_date.png')
plt.show()

For some reason, the adjusted temperature looks worse than the simple temperature feature. There seems to be gaps in the data. <font color=red>I will have to check for multicolinearity for 'temp'/'atemp' and see if there is an advantage of using 'atemp' over 'temp'. If so, I should do something about those "strange" values. Recalculate 'atemp' from other variables? Use an ML algo for that?</font>

### Humidity

In [ ]:
train['humidity_bin'] = pd.cut(train['humidity'], 10)
fig, ax = plt.subplots(2, 2, figsize=(16,9))
train['humidity'].plot(linewidth=0, marker='o', markersize=1, ax=ax[0][0])
train.boxplot(column='casual', by='humidity_bin', ax=ax[0][1], rot=45)
train.boxplot(column='registered', by='humidity_bin', ax=ax[1][0], rot=45)
train.boxplot(column='count', by='humidity_bin', ax=ax[1][1], rot=45)
plt.savefig('./../../figs/boxplot_gb_humidity.png')
plt.show()

Regarding humidity, there is also a correlation with the demand. Very dry conditions (below 10% humidity) show little demand, and this is true for all types of users. Beyond 20% humidity, the demands slowly falls but is always higher than very dry conditions. 

### Windspeed

In [ ]:
train['wind_bin'] = pd.cut(train['windspeed'], 10)
fig, ax = plt.subplots(2, 2, figsize=(16,9))
train['windspeed'].plot(linewidth=0, marker='o', markersize=1, ax=ax[0][0])
train.boxplot(column='casual', by='wind_bin', ax=ax[0][1], rot=45)
train.boxplot(column='registered', by='wind_bin', ax=ax[1][0], rot=45)
train.boxplot(column='count', by='wind_bin', ax=ax[1][1], rot=45)
plt.savefig('./../../figs/boxplot_gb_windspeed.png')
plt.show()

Finally for the windspeed feature, the demand seems pretty stable but the distribution of counts becomes more symmetric (less skewed, less outliers) at higher windspeed. As if users (casual or registered) that would use the bikes occasionnaly would not use them in strong wind conditions whereas people who tend to be very consistent using bikes still use them.

### Hour of the day

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(16,9))
train.groupby('hour')['count'].sum().plot(kind='bar', ax=ax[0][0])
train.boxplot(column='casual', by='hour', ax=ax[0][1], rot=45)
train.boxplot(column='registered', by='hour', ax=ax[1][0], rot=45)
train.boxplot(column='count', by='hour', ax=ax[1][1], rot=45)
plt.savefig('./../../figs/boxplot_gb_hour.png')
plt.show()

As we could have expected, the hour of the day is a very important feature to predict the bike demand. Also, something interesting to note here is that the profile for registered and casual users is quite different.

While registered users seem to bike for work (higher demand around 8AM, 5/6PM) and little more at lunch time, casual users have a much smoother curve with a broad maximum peaking around 2/3PM.

For this reason, it would be interesting to build <font color=red>2 separate models!</font> One to predict the number of registered users and another one for casual users. These two models will probably end up having different parameters and summing them up might result in better results than trying to predict the overall demand with a single model.

<font color='red'>One more remark: Here we groupby hours regardless of the season of the year or the day of the week for example. It is expected that if we looked at 'registered' counts for weekdays only and excluding holidays and vacations, the correlation with the target variable would be even stronger!</font>

### Day of the week

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(16,9))
train.groupby('dayofweek')['count'].sum().plot(kind='bar', ax=ax[0][0])
train.boxplot(column='casual', by='dayofweek', ax=ax[0][1], rot=45)
train.boxplot(column='registered', by='dayofweek', ax=ax[1][0], rot=45)
train.boxplot(column='count', by='dayofweek', ax=ax[1][1], rot=45)
plt.savefig('./../../figs/boxplot_gb_day.png')
plt.show()

Once again, the data behaves as expected with the demand from registered users being higher during the week and slightly lower during the weekend. For casual user, this is the other way around.

This is another good reason why two independant models should be trained for registered and casual and the individual predictions summed up.

### Here let's look at the actual distribution for week days versus weekend in the case of registered and casual separately

In [ ]:
countCasualWorkingday = train[train['workingday']==1]['casual'].sum()
countCasualWeekend = train[train['workingday']==0]['casual'].sum()

countRegisteredWorkingday = train[train['workingday']==1]['registered'].sum()
countRegisteredWeekend = train[train['workingday']==0]['registered'].sum()

print(countCasualWorkingday, countCasualWeekend, countRegisteredWorkingday, countRegisteredWeekend)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(16,9))
train[train['workingday'] == 0]['casual'].hist(ax=ax[0], alpha=0.5, color='blue', bins=10, range=(0,400), density=True, label='non-workingday')
train[train['workingday'] == 1]['casual'].hist(ax=ax[0], alpha=0.5, color='orange', bins=10, range=(0,400), density=True, label='workingday')
train[train['workingday'] == 0]['registered'].hist(ax=ax[1], alpha=0.5, color='blue', range=(0,1000), density=True, label='non-workingday')
train[train['workingday'] == 1]['registered'].hist(ax=ax[1], alpha=0.5, color='orange', range=(0,1000), density=True, label='workday')

ax[0].legend()
ax[0].set_title('Workdays and non-workdays for casual users')
ax[0].set_ylabel('relative count')

ax[1].legend()
ax[1].set_title('Workdays and non-workdays for registered users')
ax[1].set_xlabel('number of users')
ax[1].set_ylabel('relative count')

Here we have a better glimpse at the data from the previous boxplots. We can see that for casual users, the weekend usage has a much longer tail than the week usage where the demand is more peaked.
Regarding registered users, we observe the opposit behavior. The distribution for work days has a longer tail.

We can also notice that the distributions for registered users are a lot wider than the distributions for casual users.

### Month of the year

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(16,9))
train[['casual', 'registered', 'count']].resample('1M').mean().plot(ax=ax[0][0], alpha=0.5)
train.boxplot(column='casual', by='month', ax=ax[0][1], rot=45)
train.boxplot(column='registered', by='month', ax=ax[1][0], rot=45)
train.boxplot(column='count', by='month', ax=ax[1][1], rot=45)
plt.savefig('./../../figs/boxplot_gb_month.png')
plt.show()

Regarding the month of the year, we observe a similar pattern as we did with the season feature. The difference being that we have a higher granularity using the months.

Another thing to notice is the drop in "high" demand (outliers and tail of the distributions) during the months of June, July, August for casual users. This behavior is much less pronounced for registered users but still appears.
This is interesting because it correspond to a period of the year where children are on vacations and maybe, families are away from DC.

As a consequence, we should look up the school vacation calendar for 2011/2012 and create a feature 'vacation'. This could improve the predictions of our model.

### Year

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(16,9))
train[['casual', 'registered', 'count']].resample('1Y').mean().plot(ax=ax[0][0], alpha=0.5)
train.boxplot(column='casual', by='year', ax=ax[0][1], rot=45)
train.boxplot(column='registered', by='year', ax=ax[1][0], rot=45)
train.boxplot(column='count', by='year', ax=ax[1][1], rot=45)
plt.savefig('./../../figs/boxplot_gb_year.png')
plt.show()

As we already saw with the month feature, there is an increase in usage from 2011 to 2012.

The effect for casual users seems to be less pronounced (?). That might be because casual users that keeps using the bikes decide to switch to a membership (think of it as a nuturing process from the company: reach users, make them try as casual users, fidelize them and get them to sign for a membership. Then repeat.).

### Fourier Analysis of the target variables

In [ ]:
df = train['2011-05']['count'].copy()
df = df.reset_index(drop=True)

In [ ]:
f, Pxx_den = signal.periodogram(df.values.ravel())
plt.semilogy(f, Pxx_den)
plt.ylim([1e1, 1e7])
plt.xlabel('frequency [h**-1]')
plt.ylabel('PSD [V**2/Hz]')
plt.show()

# Feature engineering

### Create a cyclic variable for the hour of the day

In [ ]:
train['hour_cos'] = np.cos(2.0 * np.pi * train['hour'] / 24.0)

In [ ]:
corr_matrix = train.corr()
fig = plt.figure(figsize=(15,10))
sns.heatmap(corr_matrix, annot=True)

Strong correlation between features:
 - temp and atemp... obvious. Must do something! (pick one)
 - windspeed and humidity (-0.32)
 - humidity and weather (0.41)
 - temp/atemp and season (0.26)
 - humidity and season (0.19)
 - workingday and holiday
 
 Now with the target variable:
 - season (0.16)
 - weather (-0.13)
 - temp/atemp (0.39)
 - humidity (-0.32)
 
 <font color=red>Try to turn categorical features (season, weather) into binary features (OneHotEncoding).
    Try to use KBinarizer for continuous features (temp/atemp, humidity, windspeed)

In [ ]:
train.plot(kind='scatter', x='temp', y='atemp', marker='o')
plt.savefig('./../../figs/atemp_vs_temp.png')
plt.show()

First we can see outliers for which temp is in the range [25, 35] but atemp is always about 12.
<font color=red>If relation for atemp can be found, recalculate it myself, otherwise: remove entries with outliers or use temp.</font>

Second, there is a few points that seem to lie on a straight line: atemp = a*temp+b
<font color=red>Find the equation of the line, remove those points and thry to understand how the other points are obtained (linear regression using humidity and windspeed).</font>

Third, it seems that in the range temp = [15, 25], there is no other features explaining atemp.

## Creating a feature 'vacations'

In [ ]:
train['vacations'] = 0

train.loc['2011-04-15':'2011-04-25', 'vacations'] = 1
train.loc['2011-06-25':'2011-08-21', 'vacations'] = 1
train.loc['2011-12-22':'2012-01-02', 'vacations'] = 1
train.loc['2012-03-31':'2012-04-09', 'vacations'] = 1
train.loc['2012-06-23':'2012-08-26', 'vacations'] = 1
train.loc['2012-12-22':'2012-12-31', 'vacations'] = 1

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(16,9))
train.groupby('vacations')['count'].mean().plot(kind='bar', ax=ax[0][0])
train.boxplot(column='casual', by='vacations', ax=ax[0][1])
train.boxplot(column='registered', by='vacations', ax=ax[1][0])
train.boxplot(column='count', by='vacations', ax=ax[1][1])
plt.savefig('./../../figs/boxplot_gb_vacations.png')
plt.show()

Here it seems that the vacation feature impacts differently "different group of people". First, we see that the average demand is higher during vacations which might be due to tourists ? Also, we can notice an increase of the quantiles BUT the tail of the distributions are always shorter. The effect is more pronounced for casual users where the median almost increases by a factor of 2. Overall, the deman is higher and the distributions less skewed.

# Conclusions

Using the holiday feature, we learnt that the program is from Washington DC.

What we noticed is that despite the obvious correlation between the target variable and some of the features (months, year, hour of the day, temperature, ...) it would be hard for a linear regression model to have predictive power because there is no linear relationship between the target variable(s) and the features.
The only way to make it work would be to OneHot encode all of the features (after discretization for continuous variables) which would lead to a very large number of parameters.

Something important to keep in mind is that we should use the hour, day, month and year as predictors since the target variable(s) strongly depends on these.

There is something strange going on with the 'temp' and 'atemp' features. We never observe negative temperatures. We don't know how the 'atemp' variable is obtained and 'atemp' seems to have bad values. For the model, it will be necessary to either use 'temp' or 'atemp' after removing the bad values or correcting them. 

Finally, it seems that adding a binary feature for school vacations could improve the performances of the model.

## Check for gaps in the test data

In [ ]:
test.set_index(pd.to_datetime(test.pop('datetime')), inplace=True)

In [ ]:
test['delta_t'] = test.index.to_series().diff()

In [ ]:
test['delta_t'].fillna(pd.Timedelta('0 hour'))
test[(test['delta_t']>pd.Timedelta('1 hours')) & (test['delta_t']< pd.Timedelta('1 days'))]['delta_t']

It seems that in the test dataset, there is no entries with a total count of zero bikes. No need to create those rows to train the model.